# GraphCast Flops

## Manual Counting

### Encoder

$$
\mathbf{v}_i^{G} = \mathrm{MLP}_{\mathcal{V}^G}^{embedder}()
$$

$$
\begin{align*}
\mathbf{v}_{i}^{G} &= \text{MLP}_{\nu_G}^{\text{embedder}}\left(\mathbf{v}_{i}^{G}, \text{features}\right) \\
\mathbf{v}_{i}^{M} &= \text{MLP}_{\nu_M}^{\text{embedder}}\left(\mathbf{v}_{i}^{M}, \text{features}\right) \\
\mathbf{e}_{\nu_{s}^{M} \to \nu_{r}^{M}}^{M} &= \text{MLP}_{\mathcal{E}_M}^{\text{embedder}}\left(\mathbf{e}_{\nu_{s}^{M} \to \nu_{r}^{M}}, \text{features}\right) \\
\mathbf{e}_{\nu_{s}^{G} \to \nu_{r}^{M}}^{G2M} &= \text{MLP}_{\mathcal{E}_{G2M}}^{\text{embedder}}\left(\mathbf{e}_{\nu_{s}^{G} \to \nu_{r}^{M}}, \text{features}\right) \\
\mathbf{e}_{\nu_{s}^{M} \to \nu_{r}^{G}}^{M2G} &= \text{MLP}_{\mathcal{E}_{M2G}}^{\text{embedder}}\left(\mathbf{e}_{\nu_{s}^{M} \to \nu_{r}^{G}}, \text{features}\right)
\end{align*}
$$

$$
\begin{align*}
\mathbf{v}_{i}^{G} &= \text{MLP}_{\nu_G}^{\text{embedder}}\left(\mathbf{v}_{i}^{G}, \text{features}\right) \\
\end{align*}
$$

# Basics

## Counting MLP Flops (ptflops)

In [1]:
# import sys
# import os

# sys.path.append(os.path.dirname(os.getcwd()))

# os.chdir("../")

In [83]:
import pandas as pd


results = {
    "model": ["graph_lam", "gcn_lam", "gat_lam"],
    "dataset": ["era5_uk", "era5_uk", "era5_uk"],
    "val_rmse_z500_step1": [0, 0, 0],
    "flops": [756004160, 756004160, 756004160]
}

df = pd.DataFrame(results)

In [84]:
print(df)

       model  dataset  val_rmse_z500_step1      flops
0  graph_lam  era5_uk                    0  756004160
1    gcn_lam  era5_uk                    0  756004160
2    gat_lam  era5_uk                    0  756004160


In [1]:
import random
import time
from argparse import ArgumentParser

# Third-party
import pytorch_lightning as pl
import torch
from fvcore.nn import FlopCountAnalysis, flop_count_table
from lightning_fabric.utilities import seed

# First-party
from neural_lam import constants, utils
from neural_lam.models.graph_lam import GraphLAM
from neural_lam.models.hi_lam import HiLAM
from neural_lam.models.hi_lam_parallel import HiLAMParallel
from neural_lam.models.gcn_lam import GCNLAM
from neural_lam.models.gat_lam import GATLAM

from neural_lam.weather_dataset import WeatherDataset
from neural_lam.era5_dataset import ERA5UKDataset
from neural_lam.constants import MEPSConstants, ERA5UKConstants

MODELS = {
    "graph_lam": GraphLAM,
    "gcn_lam": GCNLAM,
    "gat_lam": GATLAM,
    "hi_lam": HiLAM,
    "hi_lam_parallel": HiLAMParallel,
}

In [18]:
def get_args():
    parser = ArgumentParser(
        description="Train or evaluate NeurWP models for LAM"
    )

    # General options
    parser.add_argument(
        "--dataset",
        type=str,
        default="meps_example",
        help="Dataset, corresponding to name in data directory "
        "(default: meps_example)",
    )
    parser.add_argument(
        "--model",
        type=str,
        default="graph_lam",
        help="Model architecture to train/evaluate (default: graph_lam)",
    )
    parser.add_argument(
        "--subset_ds",
        type=int,
        default=0,
        help="Use only a small subset of the dataset, for debugging"
        "(default: 0=false)",
    )
    parser.add_argument(
        "--seed", type=int, default=42, help="random seed (default: 42)"
    )
    parser.add_argument(
        "--n_workers",
        type=int,
        default=4,
        help="Number of workers in data loader (default: 4)",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=200,
        help="upper epoch limit (default: 200)",
    )
    parser.add_argument(
        "--batch_size", type=int, default=4, help="batch size (default: 4)"
    )
    parser.add_argument(
        "--load",
        type=str,
        help="Path to load model parameters from (default: None)",
    )
    parser.add_argument(
        "--restore_opt",
        type=int,
        default=0,
        help="If optimizer state should be restored with model "
        "(default: 0 (false))",
    )
    parser.add_argument(
        "--precision",
        type=str,
        default=32,
        help="Numerical precision to use for model (32/16/bf16) (default: 32)",
    )

    # Model architecture
    parser.add_argument(
        "--graph",
        type=str,
        default="multiscale",
        help="Graph to load and use in graph-based model "
        "(default: multiscale)",
    )
    parser.add_argument(
        "--hidden_dim",
        type=int,
        default=64,
        help="Dimensionality of all hidden representations (default: 64)",
    )
    parser.add_argument(
        "--hidden_layers",
        type=int,
        default=1,
        help="Number of hidden layers in all MLPs (default: 1)",
    )
    parser.add_argument(
        "--processor_layers",
        type=int,
        default=4,
        help="Number of GNN layers in processor GNN (default: 4)",
    )
    parser.add_argument(
        "--mesh_aggr",
        type=str,
        default="sum",
        help="Aggregation to use for m2m processor GNN layers (sum/mean) "
        "(default: sum)",
    )
    parser.add_argument(
        "--output_std",
        type=int,
        default=0,
        help="If models should additionally output std.-dev. per "
        "output dimensions "
        "(default: 0 (no))",
    )

    # Training options
    parser.add_argument(
        "--ar_steps",
        type=int,
        default=1,
        help="Number of steps to unroll prediction for in loss (1-19) "
        "(default: 1)",
    )
    parser.add_argument(
        "--control_only",
        type=int,
        default=0,
        help="Train only on control member of ensemble data "
        "(default: 0 (False))",
    )
    parser.add_argument(
        "--loss",
        type=str,
        default="wmse",
        help="Loss function to use, see metrics.py (default: wmse)",
    )
    parser.add_argument(
        "--step_length",
        type=int,
        default=3,
        help="Step length in hours to consider single time step 1-3 "
        "(default: 3)",
    )
    parser.add_argument(
        "--lr", type=float, default=1e-3, help="learning rate (default: 0.001)"
    )
    parser.add_argument(
        "--val_interval",
        type=int,
        default=1,
        help="Number of epochs training between each validation run "
        "(default: 1)",
    )
    parser.add_argument(
        "--standardize",
        type=int,
        default=1,
        help="Standardize dataset "
        "(default: 1 (true))",
    )
    parser.add_argument(
        "--overfit",
        type=int,
        default=0,
        help="Overfit to single batch for debugging "
        "(default: 0 (false))",
    )
    # Evaluation options
    parser.add_argument(
        "--eval",
        type=str,
        help="Eval model on given data split (val/test) "
        "(default: None (train model))",
    )
    parser.add_argument(
        "--n_example_pred",
        type=int,
        default=1,
        help="Number of example predictions to plot during evaluation "
        "(default: 1)",
    )
    args = parser.parse_args(args=[])
    return args

args = get_args()

args.dataset = "era5_uk"
# args.dataset = "meps_example"

args.model = "gcn_lam"
# args.model = "graph_lam"

args.graph = "uk_graphcast"

args.batch_size = 4
args.standardize = False

In [19]:
if args.dataset == "era5_uk":
    train_set = ERA5UKDataset(
        args.dataset,
        pred_length=args.ar_steps,
        split="train",
        standardize=bool(args.standardize),
    )
    val_set = ERA5UKDataset(
        args.dataset,
        pred_length=28,
        split="val",
        standardize=bool(args.standardize),
    )
    args.constants = ERA5UKConstants
elif args.dataset == "meps_example":
    train_set = WeatherDataset(
        args.dataset,
        pred_length=args.ar_steps,
        split="train",
        subsample_step=args.step_length,
        standardize=bool(args.standardize),
        subset=bool(args.subset_ds),
        control_only=args.control_only,
    )
    
    max_pred_length = (65 // args.step_length) - 2  # 19
    val_set = WeatherDataset(
        args.dataset,
        pred_length=max_pred_length,
        split="val",
        subsample_step=args.step_length,
        standardize=bool(args.standardize),
        subset=bool(args.subset_ds),
        control_only=args.control_only,
    )
    args.constants = MEPSConstants
else:
    raise ValueError(f"Unknown dataset: {args.dataset}")

model = MODELS[args.model](args)

Loaded graph with 4060 nodes (3705 grid, 355 mesh)
Edges in subgraphs: m2m=2584, g2m=7719, m2g=11094


In [20]:
init_states, target_states, forcing = train_set[0]
print("init_states", init_states.shape)
print("target_states", target_states.shape)
print("forcing", forcing.shape)

init_states torch.Size([2, 3705, 48])
target_states torch.Size([1, 3705, 48])
forcing torch.Size([1, 3705, 0])


In [21]:
input = (init_states.unsqueeze(0), target_states.unsqueeze(0), forcing.unsqueeze(0))
# out = model(input)

In [22]:
flops = FlopCountAnalysis(model, (input,))
flops.total()

RuntimeError: Cannot insert a Tensor that requires grad as a constant. Consider making it a parameter or input, or detaching the gradient
Tensor:
Columns 1 to 10-0.2114 -0.0666 -0.2028 -0.0482  0.0166  0.0643  0.1217  0.0453 -0.1695 -0.1680
-0.0906  0.1311  0.1879  0.1681 -0.0554  0.0449  0.0553  0.1052  0.1216 -0.1680
 0.0350 -0.1234 -0.0564 -0.1900  0.1943 -0.0840 -0.0252  0.1601  0.1920 -0.0926
 0.0104 -0.1734 -0.1475 -0.0902  0.0216 -0.0410  0.0569  0.1500  0.0123 -0.0974
 0.1506  0.1473  0.0959  0.1501  0.1930 -0.0254  0.1707  0.1895  0.1142 -0.0199
 0.1600 -0.0578  0.1476  0.1766  0.0522  0.1553 -0.0149 -0.0169 -0.1146 -0.2002
 0.0355  0.1792  0.1736  0.0440 -0.1987 -0.1132 -0.0099  0.1209  0.0801  0.0535
-0.1656  0.0777  0.1790  0.1804 -0.0369  0.0782 -0.1949  0.1718  0.1221  0.0780
-0.0856  0.1350 -0.1478  0.1699 -0.0423  0.1074 -0.0163  0.0923 -0.0549  0.0583
 0.2091  0.1372 -0.0725  0.0182  0.1727  0.0921  0.2064 -0.0756  0.0638 -0.1551
-0.0589  0.0591 -0.0761  0.1730 -0.1801

## Test forward pass with batch

In [33]:
train_loader = torch.utils.data.DataLoader(
    train_set,
    args.batch_size,
    shuffle=False,
    num_workers=args.n_workers,
)

batch = next(iter(train_loader))

In [34]:
batch_init_states, batch_target_states, batch_forcing = batch
print(batch_init_states.shape)
print(batch_target_states.shape)
print(batch_forcing.shape)

torch.Size([4, 2, 3705, 48])
torch.Size([4, 1, 3705, 48])
torch.Size([4, 1, 3705, 0])


In [35]:
flops = FlopCountAnalysis(model, (batch,))
flops.total()

RuntimeError: output 1 (Columns 1 to 8  4.4189   4.3034   5.7012   4.9300   4.2194   2.4090   3.8468   4.0199
  4.4001   4.2851   5.6770   4.9090   4.2015   2.3988   3.8304   4.0029
  4.3816   4.2671   5.6531   4.8884   4.1838   2.3887   3.8143   3.9860
  4.3634   4.2494   5.6296   4.8680   4.1664   2.3788   3.7985   3.9695
  4.3454   4.2319   5.6064   4.8480   4.1493   2.3690   3.7828   3.9531
  4.3277   4.2146   5.5836   4.8282   4.1324   2.3593   3.7674   3.9370
  4.3102   4.1976   5.5611   4.8088   4.1157   2.3498   3.7522   3.9211
  4.2931   4.1809   5.5389   4.7896   4.0993   2.3404   3.7373   3.9055
  4.2761   4.1644   5.5170   4.7707   4.0831   2.3312   3.7225   3.8901
  4.2594   4.1481   5.4954   4.7520   4.0672   2.3221   3.7080   3.8749
  4.2429   4.1321   5.4742   4.7337   4.0514   2.3131   3.6936   3.8599
  4.2267   4.1162   5.4532   4.7155   4.0359   2.3042   3.6795   3.8451
  4.2106   4.1006   5.4325   4.6977   4.0206   2.2955   3.6655   3.8305
  4.1948   4.0853   5.4122

In [ ]:
prediction, target, pred_std = model.common_step(batch)